<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/Cadec_twadr_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-8.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-8.validation.csv",header=None)
valid.head()

,0,1
0,13,up all night
1,16,couldn't taste anything
2,74,useless without
3,119,anxiety
4,154,urticaria


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-8.test.csv",header=None)
test.head()

,0,1
0,24,heart race 100mph
1,94,anxiety
2,154,#hivewatch
3,195,ballooning
4,195,bloated


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-cadec.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'i',
 'the',
 'and',
 'to',
 'my',
 'pain',
 'of',
 'in',
 'a',
 'it',
 'me',
 'mood',
 'for',
 'increased',
 'have',
 '-',
 'was',
 'drug',
 'this',
 ')',
 'lipitor',
 '(',
 'disorder',
 'on',
 'abnormal',
 'not',
 'withdrawal',
 'anxiety',
 'symptoms',
 'had',
 'that',
 'is',
 'with',
 'muscle',
 'weight',
 'like',
 'but',
 'taking',
 'decreased',
 'feeling',
 "n't",
 'disorders',
 'up',
 'after',
 'sleeplessness',
 'behavior',
 'syndrome',
 'sleep',
 'so',
 'all',
 'diseases',
 'got',
 'blood',
 'insomnia',
 'side',
 'feel',
 'somnolence',
 'depression',
 'effects',
 'am',
 'from',
 'finding',
 'energy',
 'loss',
 'back',
 'been',
 'disturbance',
 '!',
 'day',
 'as',
 'cholesterol',
 'skin',
 'be',
 'mental',
 'heart',
 'body',
 'gain',
 'appetite',
 'take',
 '/',
 'no',
 'at',
 'nausea',
 'acute',
 'mg',
 'has',
 'years',
 'off',
 'dizziness',
 'about',
 'fatigue',
 'now',
 'do',


In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.293756,3.433664,0.436205,08:44


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.270779,2.987772,0.496190,08:43


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.059542,2.953151,0.499524,08:41


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.035151,2.923598,0.502232,08:35


In [25]:
learn.save_encoder('TwADR-cadec_fold8_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-cadec_fold8_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.203937,6.519455,0.041667,00:25


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.282146,5.694973,0.097222,00:27


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.757297,5.392881,0.111111,01:01


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.501602,5.176967,0.138889,01:00
1,5.281579,4.847722,0.152778,01:08


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.987571,4.657434,0.166667,01:09
1,4.817712,4.480490,0.194444,01:01


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.583162,4.219368,0.256944,01:06
1,4.418577,4.131820,0.270833,01:04


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.229910,4.046194,0.284722,01:08
1,4.088981,4.001823,0.277778,01:03


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.915147,3.930950,0.270833,01:07
1,3.791461,3.834624,0.305556,01:05


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.601929,3.742095,0.326389,01:07
1,3.550718,3.736993,0.333333,01:07
2,3.357106,3.699292,0.319444,01:05
3,3.170287,3.673271,0.333333,01:09


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.039571,3.703658,0.305556,01:07
1,2.969222,3.757884,0.312500,01:07
2,2.823155,3.700092,0.340278,01:02
3,2.638332,3.680589,0.333333,01:08


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.458003,3.573875,0.347222,01:06
1,2.425415,3.611148,0.354167,01:06
2,2.262815,3.658771,0.347222,01:09
3,2.109667,3.671158,0.326389,01:06


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.992384,3.607233,0.368056,01:02
1,1.919405,3.660782,0.347222,01:08
2,1.791259,3.679759,0.354167,01:07
3,1.670478,3.670513,0.347222,01:03


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.562389,3.682163,0.340278,01:08
1,1.538171,3.809192,0.368056,01:03
2,1.415498,3.709589,0.361111,01:02
3,1.310470,3.707569,0.375000,01:07


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.197122,3.781770,0.361111,01:05
1,1.188850,3.859601,0.354167,01:05
2,1.152657,3.892955,0.368056,01:04
3,1.026380,3.828679,0.375000,01:05


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.938783,3.984104,0.347222,01:05
1,0.984722,4.123744,0.354167,01:07
2,0.910126,4.048244,0.340278,01:03
3,0.823796,4.078077,0.347222,01:08


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.786393,4.194396,0.354167,01:07
1,0.742220,4.199864,0.347222,01:10


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.738065,4.355124,0.354167,01:09
1,0.691190,4.267314,0.361111,01:05


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.667831,4.294386,0.368056,01:04


In [44]:
# save the best model

learn.save_encoder('TwADR-cadec_fold8')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.628309,4.304690,0.375000,01:06


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.600798,4.427861,0.340278,01:05


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.582550,4.418883,0.361111,01:05


# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

142
142
[1230, 978, 1356, 974, 1096, 224, 1936, 2147, 2147, 249, 1327, 1787, 1787, 1701, 1546, 1099, 1610, 1679, 265, 1227, 1227, 1810, 706, 1718, 744, 744, 749, 824, 1099, 749, 749, 1815, 792, 792, 792, 1787, 582, 1787, 1884, 1884, 265, 2010, 1136, 974, 978, 452, 749, 1089, 187, 1787, 561, 1099, 1096, 1099, 1099, 1136, 1192, 1676, 60, 1003, 1227, 1265, 1767, 1295, 1295, 1308, 1308, 1308, 1308, 1432, 1373, 1340, 1597, 1340, 1356, 1432, 749, 749, 452, 1508, 1511, 520, 2194, 1701, 978, 1062, 561, 1227, 824, 1573, 1597, 1597, 1597, 1597, 1356, 1597, 1597, 1531, 1196, 1545, 1597, 2191, 1767, 1073, 1798, 1529, 1718, 1718, 393, 2032, 1787, 1546, 1787, 393, 2032, 1815, 1610, 2149, 1936, 1546, 1940, 966, 824, 1942, 1578, 2032, 1787, 2032, 796, 189, 1295, 1767, 2103, 2113, 1184, 441, 2147, 1545, 1048, 61, 1546, 1787]
[24, 94, 154, 195, 195, 224, 233, 233, 233, 249, 371, 393, 393, 402, 418, 426, 426, 441, 445, 561, 582, 645, 706, 720, 744, 744, 749, 749, 749, 749, 749, 767, 792, 792, 792, 792, 8

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

50
0.352112676056338
